# Introduction

fdas

# Imports

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config):
    pass  # init sessin with allow_growth

# VGG Flowers 17 Dataset

In [5]:
dataset_location = '../Datasets/vgg-flowers-17/'

In [7]:
datafile = os.path.join(dataset_location, '17flowers.npz')
if not os.path.isfile(datafile):
    print('Run StanfordDogs.ipynb notebook first!')
    raise

In [8]:
npzfile = np.load(datafile)
all_images = npzfile['all_images']
all_labels = npzfile['all_labels']

In [ ]:
print(idx_to_tags.shape)
idx_to_tags[:4]

In [ ]:
train_files

In [ ]:
train_labels

In [ ]:
idx = 5678
img_path = os.path.join(dataset_location, 'images/Images', train_files[idx])
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
tag = idx_to_tags[train_labels[idx]]
plt.imshow(img)
plt.title(tag);

In [ ]:
from tensorflow.keras.applications import resnet50

In [ ]:
model = tf.keras.applications.resnet50.ResNet50(include_top=False, input_shape=(224, 224, 3))

In [ ]:
model = tf.keras.applications.resnet50.ResNet50()

In [ ]:
model.summary()

In [ ]:
img_path = os.path.join(dataset_location, 'images/Images', train_files[0])
img_path

In [ ]:
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
img.size

In [ ]:
arr = tf.keras.preprocessing.image.img_to_array(img)
arr

In [ ]:
tf.keras.applications.resnet50.preprocess_input()

In [ ]:
def load_images(folder, files, target_size):
    images_list = []
    for file in files:
        img_full_path = os.path.join(folder, file)
        img = tf.keras.preprocessing.image.load_img(img_full_path, target_size=(224,224))
        img = PIL.Image.open(img_full_path)
        img = img.resize(target_size)
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        images_list.append(np.array(img))
    return np.array(images_list)

In [ ]:
train_images = load_images(folder=os.path.join(dataset_location, 'images/Images'),
                           files=train_files, target_size=(224,224))